## Step 3 Validation: Query Engine Multi-Stage QA with Intermediate Outputs
## This notebook validates the multi-stage QA process and prints intermediate outputs:
### 1. Query decomposition (sub-queries).
### 2. Retrieval of documents for each sub-query.
### 3. Synthesis of the final answer.
### 4. Final summary with confidence rating.

### The process uses LangSmith's Client for tracing, and the retriever benefits from earlier reranking.

In [1]:
from query_engine import QueryEngine, QueryResult


import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


from langsmith import Client, traceable
from llm import get_llm
from retriever import setup_work_retriever
from text_processing import dynamic_text_splitter
from document_loader import load_work_instructions
from config import RESOURCES_PATH

# Initialize LangSmith Client for tracing.
client = Client()

/home/zamlamb/KdG/agenticrag/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/zamlamb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/zamlamb/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Extracted PDF table data: [[['Work Instruction\nDocument Code: ZF-IT-POL-R002\nDate : 14/11/2022\nVersion: 2', 'Manual Loading of Solids']], [{'1': '2', 'Ensure a vacuum of 700 mb in R-002. Place the portable hopper next to the reactor.': 'Wear the specific PPE: neoprene gloves and a face shield for the helmet.'}, {'1': '3', 'Ensure a vacuum of 700 mb in R-002. Place the portable hopper next to the reactor.': 'Remove the inlet cap from the reactor and the hose cap from the portable hopper.'}], [{'4': '5', 'Connect the hose to the reactor inlet.': 'Open the hopper lid and load the sack. Open the manual valve under the portable\nhopper to approximately 40º to allow air entry.'}, {'4': '6', 'Connect the hose to the reactor inlet.': 'Request the opening of valve H20202.'}, {'4': '7', 'Connect the hose to the reactor inlet.': 'Once the hopper is empty, request the closure of valve H20202.'}, {'4': '8', 'Connect the hose to the reactor inlet.': 'Close the hopper lid and the lower valve. Disc

In [2]:

@traceable(project_name="workstations", client=client)
def validate_query_processing(user_query: str, retriever_instance) -> QueryResult:
    # Instantiate the QueryEngine with desired parameters.
    query_engine_instance = QueryEngine(debug_mode=True, confidence_threshold=80)

    # Execute the query and get the result.
    result = query_engine_instance.query_documents_advanced(user_query, retriever_instance)
    return result

In [3]:
# Load work instructions and split into chunks.
documents = load_work_instructions(RESOURCES_PATH)
document_chunks = dynamic_text_splitter(documents, default_chunk_size=500)
print(f"Loaded {len(documents)} document(s) and created {len(document_chunks)} chunks.")

Extracted PDF table data: [[['Work Instruction\nDocument Code: ZF-IT-POL-R002\nDate : 14/11/2022\nVersion: 2', 'Manual Loading of Solids']], [{'1': '2', 'Ensure a vacuum of 700 mb in R-002. Place the portable hopper next to the reactor.': 'Wear the specific PPE: neoprene gloves and a face shield for the helmet.'}, {'1': '3', 'Ensure a vacuum of 700 mb in R-002. Place the portable hopper next to the reactor.': 'Remove the inlet cap from the reactor and the hose cap from the portable hopper.'}], [{'4': '5', 'Connect the hose to the reactor inlet.': 'Open the hopper lid and load the sack. Open the manual valve under the portable\nhopper to approximately 40º to allow air entry.'}, {'4': '6', 'Connect the hose to the reactor inlet.': 'Request the opening of valve H20202.'}, {'4': '7', 'Connect the hose to the reactor inlet.': 'Once the hopper is empty, request the closure of valve H20202.'}, {'4': '8', 'Connect the hose to the reactor inlet.': 'Close the hopper lid and the lower valve. Disc

In [4]:
# Choose the model for the Query Engine.
model_choice = "azure"  # Change to "ollama" or "qwen2.5" as needed.
llm_instance = get_llm(model_choice)


INFO:azure.identity._credentials.environment:Incomplete environment configuration for EnvironmentCredential. These variables are set: AZURE_TENANT_ID
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.0 (Linux-6.12.18-1-lts-x86_64-with-glibc2.41)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://management.azure.com/subscriptions/b98e9951-860f-464a-a9a2-f69802ca8721/resourceGroups/ai_llm/providers/Microsoft.MachineLearningServices/workspaces/agentic_rag/connections?api-version=REDACTED&category=REDACTED&includeAll=REDACTED'
Re

In [6]:
# Setup the retriever from the work chunks.
retriever_instance = setup_work_retriever(document_chunks)
# Define the two queries
queries = [
    "How do I handle hypophosphorous acid?",
    "What safety measures are required for unloading a tanker?",
    "What are the steps for HPLC Sulfated Analysis?",
    "How do I properly clean the reactor after use?",
    "What is the required PPE for handling sulfuric acid?"
]

INFO:azure.identity._credentials.environment:Incomplete environment configuration for EnvironmentCredential. These variables are set: AZURE_TENANT_ID
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'http://169.254.169.254/metadata/identity/oauth2/token?api-version=REDACTED&resource=REDACTED'
Request method: 'GET'
Request headers:
    'User-Agent': 'azsdk-python-identity/1.20.0 Python/3.12.0 (Linux-6.12.18-1-lts-x86_64-with-glibc2.41)'
No body was attached to the request
INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://management.azure.com/subscriptions/b98e9951-860f-464a-a9a2-f69802ca8721/resourceGroups/ai_llm/providers/Microsoft.MachineLearningServices/workspaces/agentic_rag/connections?api-version=REDACTED&category=REDACTED&includeAll=REDACTED'
Re

In [7]:
# Display the final output with full metadata for each source document.
# Initialize the QueryEngine with debug mode for detailed output
for query in queries:
    query_result = validate_query_processing(query, retriever_instance)
    print("=" * 80)
    print(f"Processing Query: {query}")
    print("=" * 80)
    print("Final Answer:", query_result.answer)
    print("Confidence:", query_result.confidence, "%")
    print("Source Documents Metadata:")
    for source in query_result.source_documents:
        print("-" * 40)
        for key, value in source.items():
            print(f"{key}: {value}")

/home/zamlamb/KdG/agenticrag/llm.py:80: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  self._ollama = Ollama(model=model, temperature=temperature)
/home/zamlamb/KdG/agenticrag/llm.py:87: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self._ollama(prompt, stop=stop)
/home/zamlamb/KdG/agenticrag/query_engine.py:269: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = qa_chain_instance.retriever.get_relevant_documents(subq)
INFO:httpx:HTTP Request: PO

Processing Query: How do I handle hypophosphorous acid?
Final Answer: To handle hypophosphorous acid, follow these steps:
* **Loading Process:**
1. Open manual valve 1 for loading 50% hypophosphorous acid, located above R-001.
2. Ensure the reactor is under vacuum.
3. Remove the cap from the acid container and insert Cane A.
* **Container Management:**
1. If the container is empty, place it in the exit area.
2. From the entry area, take a new container and place it on the scale.
3. The field operator is responsible for collecting empty containers and ensuring a spare container is available in the entry area.
Confidence: 95%
Confidence: 95 %
Source Documents Metadata:
----------------------------------------
format: PDF 1.7
title: 
author: Ricard Torralba
subject: 
keywords: 
creator: Microsoft® Word for Microsoft 365
producer: Microsoft® Word for Microsoft 365
creationDate: D:20250204224823+01'00'
modDate: D:20250204224823+01'00'
trapped: 
encryption: None
Document Name: Work Instructi

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/work_instructions_RAG/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/work_instructions_RAG/points/search "HTTP/1.1 200 OK"
INFO:query_engine:Retrieved document content: Deliver the sample to the control laboratory along with the analysis
8 report. Wait for the quality OK before starting the unloading process. Driver: Using a lifeline, harness, and safety guardrail, open the bottom valve. 9
Open the tanker outlet valve and the manual line valve. Select the unloading tank and request plant authorization. Start the pump. 10
During unloading, the driver must remain outside the cab and in view. Check and complete tanker documentation. Monitor for leaks and tank leve
INFO:query_engine:Retrieved document metadata: {'format

Processing Query: What safety measures are required for unloading a tanker?
Final Answer: To ensure safety during the unloading process, the following measures should be taken:
* Verify the unloading order details, including the unloading tank, product, and correct level for unloading.
* Deliver a sample to the control laboratory along with the analysis report and wait for quality confirmation (OK) before starting the unloading process.
* Use specific Personal Protective Equipment (PPE) for operators and drivers based on product types, such as neoprene gloves, face shields, and aprons.
* During the unloading process, use a harness, lifeline, and guardrail while opening or closing valves like the bottom valve or hatches.
* At the end of unloading, verify that the tanker is completely empty.
* Close all relevant valves: bottom valve, hatches, and outlet valve.
Confidence: 95%
Confidence: 95 %
Source Documents Metadata:
----------------------------------------
format: PDF 1.7
title: 
auth

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/work_instructions_RAG/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/work_instructions_RAG/points/search "HTTP/1.1 200 OK"
INFO:query_engine:Retrieved document content: AG Solution Spain S.A.U. C. Pujades 350, Planta 4, Puerta 1
08019 Barcelona – Spain
www.agsolutiongroup.com
+34 93 624 02 75
ES A 65931651
Work Instruction
Document Code: ZF-IT-SUL-
Laboratory Sulfated Analysis by HPLC
Date : 31/7/2020
Version: 1
Puncture the sample. Open the slot to insert the needle and introduce the sample. Remove the needle and close the slot. 1
Enter the Sample Name and in Method select the corresponding base for the analysis
to be performed. Press Inject.
INFO:query_engine:Retrieved document metadata: {'format': 'PDF 1.7', 'tit

Processing Query: What are the steps for HPLC Sulfated Analysis?
Final Answer: Steps for HPLC Sulfated Analysis:
* Puncture the sample.
* Open the slot to insert the needle and introduce the sample.
* Remove the needle and close the slot.
Confidence: 100 %
Source Documents Metadata:
----------------------------------------
format: PDF 1.7
title: 
author: Ricard Torralba
subject: 
keywords: 
creator: Microsoft® Word for Microsoft 365
producer: Microsoft® Word for Microsoft 365
creationDate: D:20250206231231+01'00'
modDate: D:20250206231231+01'00'
trapped: 
encryption: None
Document Name: Work Instruction - Sulfated Analysis by HPLC
Step Number: None
Author: Ricard Torralba
Created Date: D:20250206231231+01'00'
Keywords: ['']
Process Type: None
Safety Measures: []
Version: 1
Date: 31/7/2020
_id: d9bb1e6e-a76f-4568-9f44-a87b65792e44
_collection_name: work_instructions_RAG
----------------------------------------
metadata: {}
_id: 1aca078f-c1f4-4b0c-8562-7bd7b79679e0
_collection_name: work

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/work_instructions_RAG/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/work_instructions_RAG/points/search "HTTP/1.1 200 OK"
INFO:query_engine:Retrieved document content: Slowly open manual valve 2 to load hypophosphorous acid into R-001. The
3 amount loaded into the reactor is controlled by the weight difference on the scale. Once the desired amount has been loaded, lift the cane and allow air to be drawn in
until it is clean.
INFO:query_engine:Retrieved document metadata: {'format': 'PDF 1.7', 'title': '', 'author': 'Ricard Torralba', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word for Microsoft 365', 'producer': 'Microsoft® Word for Microsoft 365', 'creationDate': "D:20250204224823+01'00'", 'modDate': "D

Processing Query: How do I properly clean the reactor after use?
Final Answer: * Disconnect the hose and place the caps back on the reactor inlet and hose.
* Wear specific PPE: neoprene gloves and a face shield for the helmet.
* Ensure a vacuum of 700 mb in R-002 before cleaning.
* Follow proper procedures to ensure thorough cleaning as instructed.
Confidence: 85%
Confidence: 85 %
Source Documents Metadata:
----------------------------------------
format: PDF 1.7
title: 
author: Ricard Torralba
subject: 
keywords: 
creator: Microsoft® Word for Microsoft 365
producer: Microsoft® Word for Microsoft 365
creationDate: D:20250206230521+01'00'
modDate: D:20250206230521+01'00'
trapped: 
encryption: None
Document Name: Work Instruction - Manual Load Solids
Step Number: None
Author: Ricard Torralba
Created Date: D:20250206230521+01'00'
Keywords: ['']
Process Type: None
Safety Measures: []
Version: 2
Date: 14/11/2022
_id: fca9d12f-50c2-47eb-b855-233b61d50c8f
_collection_name: work_instructions_R

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/work_instructions_RAG/points/search "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/work_instructions_RAG/points/search "HTTP/1.1 200 OK"
INFO:query_engine:Retrieved document content: AG Solution Spain S.A.U. C. Pujades 350, Planta 4, Puerta 1
08019 Barcelona – Spain
www.agsolutiongroup.com
+34 93 624 02 75
ES A 65931651
Work Instruction
Document Code: ZF-IT-POL-R001-01
Hypophosphorous Acid Addition
Date : 20/06/2020
Version: 2
Wear the specific PPE: neoprene gloves, face shield for the helmet, and an apron. 1
Open manual valve 1 for loading 50% hypophosphorous acid, located above R-001. 2
With the reactor under vacuum, remove the cap from the acid container and insert
Cane A
INFO:query_engine:Retrieved document metadata: {'format

Processing Query: What is the required PPE for handling sulfuric acid?
Final Answer: * Required PPE for handling sulfuric acid includes:
+ Neoprene gloves
+ Face shield
+ Apron
Confidence: 100%
Confidence: 100 %
Source Documents Metadata:
----------------------------------------
format: PDF 1.7
title: 
author: Ricard Torralba
subject: 
keywords: 
creator: Microsoft® Word for Microsoft 365
producer: Microsoft® Word for Microsoft 365
creationDate: D:20250204224823+01'00'
modDate: D:20250204224823+01'00'
trapped: 
encryption: None
Document Name: Work Instruction - Hypophosphorous Acid
Step Number: None
Author: Ricard Torralba
Created Date: D:20250204224823+01'00'
Keywords: ['']
Process Type: None
Safety Measures: []
Version: 2
Date: 20/06/2020
_id: dc293ee1-4c41-481b-a387-d94ec1efb0f0
_collection_name: work_instructions_RAG
----------------------------------------
format: PDF 1.7
title: 
author: Ricard Torralba
subject: 
keywords: 
creator: Microsoft® Word for Microsoft 365
producer: Micr